In [ ]:
import re
import string
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the datasets
def load_data(train_path, valid_path, test_path):
    train_data = pd.read_json(train_path)
    valid_data = pd.read_json(valid_path)
    test_data = pd.read_json(test_path)
    return train_data, valid_data, test_data

train_data, valid_data, test_data = load_data(
    '/content/sample_data/sarcdatatrain.json',
    '/content/sample_data/sarcdatavalid.json',
    '/content/sample_data/sarcdatatest.json'
)

In [ ]:

# Combine train and validation datasets for training
data = pd.concat([train_data, valid_data], ignore_index=True)


In [ ]:
# Clean text function
def clean_text(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'https?://\S+|www\.\S+', '', sentence)  # Remove URLs
    sentence = re.sub(r'<.*?>', '', sentence)  # Remove HTML tags
    sentence = re.sub(r'[%s]' % re.escape(string.punctuation), '', sentence)  # Remove punctuation
    sentence = re.sub(r'\n', '', sentence)  # Remove newlines
    return sentence


In [ ]:

# Apply cleaning
data['cleaned_text'] = data['text'].apply(clean_text)
test_data['cleaned_text'] = test_data['text'].apply(clean_text)


In [ ]:

# Load BERT tokenizer and model also ignoring case sensitvity
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Function to encode texts
def encode_texts(texts):
    return tokenizer(
        texts.tolist(),#converts into list
        max_length=64, #64 tokens
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

encoded_train = encode_texts(data['cleaned_text'])
encoded_test = encode_texts(test_data['cleaned_text'])


In [ ]:
# Train-test split
X_train = encoded_train['input_ids']
X_test = encoded_test['input_ids']
attention_train = encoded_train['attention_mask']
attention_test = encoded_test['attention_mask']
y_train = data['label'].values
y_test = test_data['label'].values


In [ ]:
# Build the model
input_ids = tf.keras.Input(shape=(64,), dtype=tf.int32, name='input_ids')
attention_masks = tf.keras.Input(shape=(64,), dtype=tf.int32, name='attention_mask')

bert_output = bert_model([input_ids, attention_masks])
x = bert_output.pooler_output #pooler_output: A 768-dimensional vector summarizing the input.
x = tf.keras.layers.Dense(128, activation='relu')(x) #desnse layer: learns intermediate patterns
x = tf.keras.layers.Dropout(0.3)(x) # prevents overfitting
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    [X_train, attention_train], y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=32,
)


Epoch 1/5
618/618 [==============================] - 250s 404ms/step - loss: 0.0332 - accuracy: 0.9891 - val_loss: 0.7566 - val_accuracy: 0.8310
Epoch 2/5
618/618 [==============================] - 248s 401ms/step - loss: 0.0270 - accuracy: 0.9902 - val_loss: 0.8513 - val_accuracy: 0.8355
Epoch 3/5
618/618 [==============================] - 248s 401ms/step - loss: 0.0206 - accuracy: 0.9929 - val_loss: 0.9036 - val_accuracy: 0.8314
Epoch 4/5
618/618 [==============================] - 248s 401ms/step - loss: 0.0191 - accuracy: 0.9933 - val_loss: 0.8950 - val_accuracy: 0.8251
Epoch 5/5
618/618 [==============================] - 248s 401ms/step - loss: 0.0180 - accuracy: 0.9937 - val_loss: 0.9577 - val_accuracy: 0.8310


In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate([X_test, attention_test], y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
# Plot accuracy and loss
def plot_graph.s(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric])
    plt.xlabel("Epochs")
    plt.ylabel(metric.capitalize())
    plt.legend([metric.capitalize(), 'Val_' + metric.capitalize()])
    plt.show()


In [ ]:
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')


In [ ]:
# Confusion Matrix
y_pred = (model.predict([X_test, attention_test]) > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# Classification Report
print(classification_report(y_test, y_pred))


In [ ]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:

# Sentiment Analysis Pipeline
sentiment_pipeline = pipeline("sentiment-analysis")


In [ ]:
# Function for user input and predictions
def predict_sarcasm_and_sentiment():
    input_sentence = input("Enter a sentence: ")
    cleaned_sentence = clean_text(input_sentence)

    # Sarcasm Prediction
    encoded_input = tokenizer(
        [cleaned_sentence],
        max_length=64,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    sarcasm_prediction = model.predict([encoded_input['input_ids'], encoded_input['attention_mask']])
    if sarcasm_prediction[0][0] > 0.5:
        print("Sarcasm Detection: Sarcastic")
    else:
        print("Sarcasm Detection: Not Sarcastic")

    # Sentiment Analysis
    sentiment_result = sentiment_pipeline(input_sentence)
    print(f"Sentiment Analysis: {sentiment_result[0]['label']}")


In [ ]:

# Run prediction
predict_sarcasm_and_sentiment()
